In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import signal
import os
import time
import sys

save_path = '/content/translation_model_v2.keras'

def signal_handler(sig, frame):
    print('You pressed Ctrl+C!')
    model.save(save_path)
    print(f'Model saved to: {save_path}')
    print('Exiting...')
    time.sleep(1)
    try:
       sys.exit(0)
    except SystemExit:
       print("Exiting training...")
       raise


global_batch_size = 256


# Rest of your code
eng_sent = []
frn_sent = []

df = pd.read_csv("/content/eng_-french.csv")
eng_sent = df.iloc[:, 0].tolist()
frn_sent = df.iloc[:, 1].tolist()

tokenizer_eng = Tokenizer()
tokenizer_frn = Tokenizer()

tokenizer_eng.fit_on_texts(eng_sent)
eng_sequences = tokenizer_eng.texts_to_sequences(eng_sent)
eng_word_index = tokenizer_eng.word_index

tokenizer_frn.fit_on_texts(frn_sent)
frn_sequences = tokenizer_frn.texts_to_sequences(frn_sent)
frn_word_index = tokenizer_frn.word_index

eng_padded = pad_sequences(eng_sequences)
frn_padded = pad_sequences(frn_sequences)


# Train-test split
eng_train, eng_test, frn_train, frn_test = train_test_split(eng_padded, frn_padded, test_size=0.2, random_state=42)

In [4]:
if os.path.exists(save_path):
    # Load the model if it exists
    model = tf.keras.models.load_model(save_path)
    print(f"Model loaded from: {save_path}")
else:
  model = tf.keras.Sequential(
      [
          tf.keras.layers.Embedding(
              input_dim=len(eng_word_index) + 1,
              output_dim=512,  # Increased embedding dimension
              input_length=eng_padded.shape[1],
          ),
          tf.keras.layers.LSTM(512),  # Increased LSTM units
          tf.keras.layers.RepeatVector(frn_padded.shape[1]),
          tf.keras.layers.LSTM(512, return_sequences=True),  # Increased LSTM units
          tf.keras.layers.Dropout(0.2),  # Added dropout for regularization
          tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(frn_word_index) + 1, activation="softmax")),
      ]
  )




  model.compile(
      optimizer="adam",
      loss="sparse_categorical_crossentropy",
      metrics=["accuracy"],
  )

  print("Model created")

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)  # Adjust patience as needed
signal.signal(signal.SIGINT, signal_handler)


# history = model.fit(
#     eng_train,
#     np.expand_dims(frn_train, -1),
#     epochs=10,
#     batch_size=global_batch_size,
#     validation_split=0.2,
#     callbacks=[reduce_lr, early_stopping]
# )



# Evaluate the model
loss, accuracy = model.evaluate(eng_test, np.expand_dims(frn_test, -1))
print(f"Test Accuracy: {accuracy}")

Model loaded from: /content/translation_model_v2.keras
1098/1098 ━━━━━━━━━━━━━━━━━━━━ 93s 84ms/step - accuracy: 0.8781 - loss: 1.2073
Test Accuracy: 0.8779575228691101


In [3]:
!pip install scikit-optimize
from skopt import gp_minimize
from skopt.space import Real, Integer, Categorical
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import signal
import os
import time
import sys
space  = [
    Integer(128, 512, name='embedding_dim'),
    Integer(128, 512, name='lstm_units'),
    Real(0.1, 0.5, name='dropout_rate'),
    Integer(32, 256, name='batch_size'),
]

def objective_function(params):
    # Unpack hyperparameters
    embedding_dim, lstm_units, dropout_rate, batch_size = params

    # Ensure lstm_units is an integer
    lstm_units = int(lstm_units)  # Convert to integer if necessary

    save_path = os.path.join('/content', f'translation_model_embed_{embedding_dim}_lstm_{lstm_units}_drop_{dropout_rate:.2f}_batch_{batch_size}.keras')


    if os.path.exists(save_path):
      # Load the model if it exists
      model = tf.keras.models.load_model(save_path)
      print(f"Model loaded from: {save_path}")
    else:
      # Build the model with the given hyperparameters
      model = tf.keras.Sequential([
          tf.keras.layers.Embedding(input_dim=len(eng_word_index) + 1,
                                    output_dim=embedding_dim,
                                    input_length=eng_padded.shape[1]),
          tf.keras.layers.LSTM(lstm_units),
          tf.keras.layers.RepeatVector(frn_padded.shape[1]),
          tf.keras.layers.LSTM(lstm_units, return_sequences=True),
          tf.keras.layers.Dropout(dropout_rate),
          tf.keras.layers.TimeDistributed(tf.keras.layers.Dense(len(frn_word_index) + 1, activation="softmax")),
      ])

      model.compile(
          optimizer="adam",
          loss="sparse_categorical_crossentropy",
          metrics=["accuracy"],
          )

      print("Model created")
      print(f"Hyperparameters: embedding_dim={embedding_dim}, lstm_units={lstm_units}, dropout_rate={dropout_rate}, batch_size={batch_size}")

    # Train the model
    history = model.fit(
        eng_train,
        np.expand_dims(frn_train, -1),
        epochs=2,  # You might adjust the number of epochs
        batch_size=batch_size,
        validation_split=0.2,
        )

    model.save(save_path)
    print(f"Model saved to: {save_path}")

    # Evaluate the model
    loss, accuracy = model.evaluate(eng_test, np.expand_dims(frn_test, -1))
    print(f"Test Accuracy: {accuracy}")

    # Return the validation accuracy (negative to minimize)
    return -history.history['val_accuracy'][-1]

result = gp_minimize(objective_function, space, n_calls=50, random_state=42)
# n_calls controls the number of iterations
best_params = result.x
print("Best Hyperparameters:", best_params)

Model created
Hyperparameters: embedding_dim=434, lstm_units=198, dropout_rate=0.41187640010910787, batch_size=166
Epoch 1/2


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


263/678 ━━━━━━━━━━━━━━━━━━━━ 3:26 497ms/step - accuracy: 0.8572 - loss: 3.2493

KeyboardInterrupt: 